[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [2]:
import os, getpass

# Calls setting environment variable for this specific key
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [3]:
# Import Statements
import os, getpass

# Check and retrieve the environment variable directly
def get_api_key(var: str):
    # Assuming the environment variable is already set
    api_key = os.environ.get(var)
    
    if api_key:
        return api_key
    else:
        raise EnvironmentError(f"{var} not set in the environment.")

api_key = get_api_key("OPENAI_API_KEY")

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-do for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [4]:
# We are using ChatOPENAI
from langchain_openai import ChatOpenAI

# Initliaze both GPT versions
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Temperature = 0 factual responses
# Closer to 1 are better for more various responses

# We will primary use stream and invoke.
# Stream - parts, or tokens, of the "response" from OpenAI's model is returned
# Invoke - call and return the full payload, full thing

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [5]:
# HumanMessage - function needed to pass Human Message -> Model Language Message
from langchain_core.messages import HumanMessage

# Create a message, tell the Chat Model your name
msg = HumanMessage(content="What is the country capital of Canada?", name="Christine")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
result = gpt4o_chat.invoke(messages)

# Notice the output has several things

In [29]:
result

AIMessage(content='The capital of Canada is Ottawa.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 17, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-df97ce19-653e-409c-af16-428579ffa398-0', usage_metadata={'input_tokens': 17, 'output_tokens': 7, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
# Pass a human message, but you can pass a string directly
# without "HumanMessage"

# Stream, Invoke - same for all the existing LLM Chat Models

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9c6d137-7c1d-4e37-a850-9f4f1d2331cb-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [9]:
a = gpt35_chat.stream("hello world")

In [10]:
for item in a:
    print(item)

content='' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content='Hello' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content='!' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' How' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' can' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' I' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' assist' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' you' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content=' today' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-406b-932f-38791030f91f'
content='?' additional_kwargs={} response_metadata={} id='run-3197226c-6c43-

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [11]:
_set_env("TAVILY_API_KEY")

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("How are you feeling?")

In [15]:
search_docs

[{'url': 'https://en.wikipedia.org/wiki/2%2B2',
  'content': 'Look up two plus two equals five in Wiktionary, the free dictionary. 2+2 or Two plus two may refer to: Four, the result of the expression 2+2\xa0...'},
 {'url': 'https://www.tiktok.com/@harrisay15/video/7343706642715774240?lang=en',
  'content': 'Mar 7, 2024 · What Is Half of 2 Plus 2. Understanding the Meaning of 2 + 2 | Fun Interview with ...Duration: 0:31Posted: Mar 7, 2024'},
 {'url': 'https://www.tiktok.com/discover/whats-2-2-meme?lang=en',
  'content': "8 days ago · What is 2 plus two? I don't get. I don't get it. I. don't. get it. Damn! What is the meaning of 2 + 2? I don't understand. See, in Japanese."}]